In [16]:
%load_ext autoreload
%autoreload 2

from tweedejaars_project import *
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder
import xgboost
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

df = load_df()

print(df)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
                        datetime  import_capacity  upward_dispatch_published  \
0      2023-12-29 00:00:00+00:00             65.4                        0.0   
1      2023-12-29 00:01:00+00:00             65.4                        0.0   
2      2023-12-29 00:02:00+00:00             65.4                        0.0   
3      2023-12-29 00:03:00+00:00             65.4                        0.0   
4      2023-12-29 00:04:00+00:00             65.4                        0.0   
...                          ...              ...                        ...   
220241 2024-05-29 22:41:00+00:00           1982.6                      167.0   
220242 2024-05-29 22:42:00+00:00           1982.6                      165.0   
220243 2024-05-29 22:43:00+00:00           1982.6                      162.0   
220244 2024-05-29 22:44:00+00:00           1982.6                      158.0   
220245 2024-05-29 22:45:00+00:00

In [17]:
features = [
    "import_capacity",
    # "min_price_published",
    "mid_price_published",
    # "max_price_published",
    # "upward_dispatch_published",
    # "downward_dispatch_published",
    # 'min_ptu_price_known',
    # "max_ptu_price_known",
    # "settlement_price_bestguess",
    'PTU',
    # 'forecast_wind',
    # 'forecast_solar',
    # 'forecast_demand',
    'time_since_last_two_sided',
    'two_sided_daily_count',
]

In [18]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Combine features into a single dataset
X = df[features].values
y = df['target_two_sided_ptu'].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

# Separate the training data by class
X_train_class0 = X_train[y_train == 0]
X_train_class1 = X_train[y_train == 1]

# Train an HMM for each class
hmm_class0 = hmm.GaussianHMM(n_components=3, covariance_type="diag", n_iter=100)
hmm_class0.fit(X_train_class0)

hmm_class1 = hmm.GaussianHMM(n_components=3, covariance_type="diag", n_iter=100)
hmm_class1.fit(X_train_class1)

# Define a function for classification
def classify_hmm(hmm_class0, hmm_class1, X):
    # Reshape X if it's 1-dimensional
    if X.ndim == 1:
        X = X.reshape(1, -1)  # Reshape to (1, m)

    log_likelihood_class0 = hmm_class0.score(X)
    log_likelihood_class1 = hmm_class1.score(X)

    return 0 if log_likelihood_class0 > log_likelihood_class1 else 1

# Evaluate on the test set
predictions = []
for sequence in X_test:
    predictions.append(classify_hmm(hmm_class0, hmm_class1, sequence))

accuracy = np.mean(predictions == y_test)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9266


In [24]:
print(predictions)

0        0
1        0
2        0
3        0
4        0
        ..
66069    0
66070    0
66071    0
66072    0
66073    0
Length: 66074, dtype: int64
